При обучении >15 эпох, качество ответов модели падает, все ответы становятся однообразными.\

TODO Увеличить количество ингредиентов для каждого рецепта. В данным момент - расчет на порцию => Генерируемые рецепты содержат слишком малое их количество

In [1]:
from scripts.custom_dataset import CustomDataset
from scripts.vectorizer import Seq2Seq_Vectorizer
from scripts.tokenizer import SeparatorTokenizer
from scripts.vocabulary import Vocabulary
from scripts.model import TransformerModel, subsequent_mask

import os
import math
import time
import json
import pandas
import pandas as pd
import regex as re
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
TEST_PROPORTION = 0.0
EVAL_PROPORTION = 0.0

TOKENS_TRESHOLD_FREQ = 10

SHUFFLE = True
DROP_LAST = True
EPOCHS = 0
LEARNING_RATE = 0.00001

LR_SCHEDULER_FACTOR = 0.5
LR_SCHEDULER_PATIENCE = 2

USE_PRETRAINED = True

BATCH_SIZE = 16
EMBEDDING_DIM = 128
MODEL_DIM = 384
NUM_HEAD = 8
NUM_ENCODER_LAYERS = 4
NUM_DECODER_LAYERS = 6
FC_HIDDEN_DIM = MODEL_DIM*4 # Как в классическом трансформере
DROPOUT = 0.1
TEMPERATURE = 0.7
BATCH_FIRST = True

MAX_SOURCE_SEQ_LEN = 30 # Максимальная длина берется из датафрейма
MAX_TARGET_SEQ_LEN = 1300
MAX_SEQ_LEN = max(MAX_SOURCE_SEQ_LEN, MAX_TARGET_SEQ_LEN)

MODEL_SAVE_FILEPATH = 'data/model_params_recipes.pt'
DATASET_FILEPATH = 'D:/Files/Datasets/recipes_generation/all_recepies_preprocessed.csv'

RANDOM_STATE = 42

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
print(DEVICE)

cuda


In [4]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'):
    '''
    Создает батчи из датасета и переносит данные на указанное устройство.
    
    Args:
        dataset (Dataset): Датасет для создания батчей
        batch_size (int): Размер батча
        shuffle (bool): Перемешивать ли данные
        drop_last (bool): Отбрасывать ли последний неполный батч
        device (str): Устройство для переноса данных ('cpu' или 'cuda')
    
    Yields:
        dict: Словарь с тензорами батча, перенесенными на устройство
    '''
    dataloader = DataLoader(dataset, batch_size, shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [5]:
def normalize_sizes(y_pred:torch.tensor, y_true:torch.tensor):
    '''
    Нормализует размеры тензоров предсказаний и целевых значений.
    
    Args:
        y_pred (torch.Tensor): Тензор предсказаний модели (3D или 2D)
        y_true (torch.Tensor): Тензор целевых значений (2D или 1D)
    
    Returns:
        tuple: Нормализованные тензоры (y_pred, y_true)
    '''
    if len(y_pred.size()) == 3:
        y_pred = y_pred.reshape(-1, y_pred.size(2))
    if len(y_true.size()) == 2:
        y_true = y_true.reshape(-1)
    return y_pred, y_true

In [6]:
def compute_accuracy(y_pred, y_true, mask_index):
    '''
    Вычисляет точность предсказаний модели, игнорируя маскированные токены.
    
    Args:
        y_pred (torch.Tensor): Тензор предсказаний модели
        y_true (torch.Tensor): Тензор целевых значений
        mask_index (int): Индекс маскированного токена
    
    Returns:
        float: Значение точности в процентах
    '''
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.max(dim=1)
    
    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()
    
    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100

In [7]:
def sequence_loss(y_pred, y_true, mask_index):
    '''
    Вычисляет функцию потерь для последовательностей с игнорированием маскированных токенов.
    
    Args:
        y_pred (torch.Tensor): Тензор предсказаний модели
        y_true (torch.Tensor): Тензор целевых значений
        mask_index (int): Индекс маскированного токена
    
    Returns:
        torch.Tensor: Значение функции потерь
    '''
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

In [8]:
def get_tokens_freq(dataframe: pandas.DataFrame) -> tuple[dict, dict]:
    '''
    Вычисляет частоту встречаемости токенов в датафрейме.
    
    Args:
        dataframe (pandas.DataFrame): Датафрейм с токенизированными текстами
    
    Returns:
        dict: Словарь с частотой токенов
    '''
    tokens_freq = {}
    for i in range(len(dataframe)):
        source_tokens, target_tokens = (dataframe.loc[i, 'source_text'], dataframe.loc[i, 'target_text'])
        for token in source_tokens:
            if token in tokens_freq:
                tokens_freq[token] += 1
            else:
                tokens_freq[token] = 1
        for token in target_tokens:
            if token in tokens_freq:
                tokens_freq[token] += 1
            else:
                tokens_freq[token] = 1
    return tokens_freq

In [9]:
def get_max_tokenized_seq_len(dataframe: pandas.DataFrame) -> tuple[int, int]:
    '''
    Находит максимальную длину исходных и целевых последовательностей в датафрейме.
    
    Args:
        dataframe (pandas.DataFrame): Датафрейм с токенизированными текстами
    
    Returns:
        tuple: (макс. длина исходных текстов, макс. длина целевых текстов)
    '''
    source_max_len = target_max_len = -1
    for idx in range(len(dataframe)):
        source_max_len = max(len(dataframe.loc[idx, 'source_text']), source_max_len)
        target_max_len = max(len(dataframe.loc[idx, 'target_text']), target_max_len)
    return source_max_len, target_max_len

In [10]:
# TODO доделать генерацию нескольких ответов за запуск
def generate(model, tokenizer, vectorizer, query: str, max_seq_len: int = 100, 
             seq_cnt: int = 1, temperature: float = 1.0, device: str = 'cpu'):
    '''
    Генерирует последовательность на основе входного запроса с помощью модели.
    
    Args:
        model (TransformerModel): Обученная модель трансформера
        tokenizer (SeparatorTokenizer): Токенизатор для обработки текста
        vectorizer (Seq2Seq_Vectorizer): Векторизатор для преобразования токенов
        query (str): Входной запрос для генерации
        max_seq_len (int): Максимальная длина генерируемой последовательности
        seq_cnt (int): Количество генерируемых последовательностей
        temperature (float): Температура для генерации (контроль случайности)
        device (str): Устройство для вычислений
    
    Returns:
        torch.Tensor: Тензор с сгенерированными последовательностями индексов токенов
    '''
    model.to(device)
    model.eval()

    bos_index = vectorizer.tokens_vocab._bos_index
    eos_index = vectorizer.tokens_vocab._eos_index
    source_mask_index = vectorizer.tokens_vocab.mask_token_index
    target_mask_index = vectorizer.tokens_vocab.mask_token_index

    tokenized_lst = tokenizer.tokenize(query)
    vectorized_dict = vectorizer.vectorize(source_tokens=tokenized_lst, use_dataset_max_len=False)

    source = torch.tensor(vectorized_dict['source_vec'], dtype=torch.int).to(device).unsqueeze(0)
    source = source.expand(seq_cnt, -1) # Расширяем для генерации нескольких ответов

    embeded = model.source_embedding(source)
    pos_embeded = model.pos_encoding_encoder(embeded) * math.sqrt(model.embed_dim)
    source_embed_projected = model.embed_to_model_projection(pos_embeded)
    source_key_padding_mask = (source == source_mask_index).to(device)

    # Проход через энкодер
    encoder_output = model.transformer.encoder(source_embed_projected, src_key_padding_mask=source_key_padding_mask)

    # Инициализация decoder_input <BOS> токеном
    bos_list = [[bos_index]] * seq_cnt
    decoder_input = torch.tensor(bos_list, device=device, dtype=torch.int)
        
    # Пошаговая генерация последовательности
    for _ in range(max_seq_len):
        # Проход через декодер

        target_embed = model.target_embedding(decoder_input)
        target_embed = model.pos_encoding_decoder(target_embed) * math.sqrt(model.embed_dim)
        target_embed_projected = model.embed_to_model_projection(target_embed)
            
        decoder_output = model.transformer.decoder(
            target_embed_projected,
            encoder_output,
            tgt_mask=subsequent_mask(decoder_input.size(1), device=device),
            tgt_key_padding_mask=(decoder_input == target_mask_index),
            memory_key_padding_mask=source_key_padding_mask)

        # Получение предсказания следующего токена
        logits = model.classifier(decoder_output[:, -1, :])
        probs = F.softmax(logits/temperature, dim=-1)
        next_token = torch.multinomial(probs, 1)

        # Добавление нового токена к последовательности
        decoder_input = torch.cat([decoder_input, next_token], dim=1)
            
        # Проверка на окончание последовательности
        if next_token.size(0) == 1:
            if next_token.item() == eos_index:
                return decoder_input

    return decoder_input

In [11]:
def decode_indices(indices: torch.tensor, vectorizer):
    '''
    Декодирует тензор индексов обратно в текстовое представление.
    
    Args:
        indices (torch.Tensor): Тензор с индексами токенов
        vectorizer (Seq2Seq_Vectorizer): Векторизатор для обратного преобразования
    
    Returns:
        list: Список декодированных строк
    '''
    seq_count, seq_len = (indices.size(0), indices.size(1))
    vocab = vectorizer.tokens_vocab
    decoded = []
    for seq in range(seq_count):
        string =''
        for idx in range(seq_len):
            index = indices[seq, idx].item()
            if index != vocab.mask_token_index:
                string += vocab.get_token(index) + ' '
            if index == vocab._eos_index:
                break
        decoded.append(string)
    return decoded

In [12]:
def save_results_to_file(model, model_filepath:str, train_states:list, validation_states:list):
    '''
    Сохраняет параметры модели и метрики обучения в файлы.
    
    Args:
        model (TransformerModel): Обученная модель
        model_filepath (str): Путь для сохранения модели
        train_states (list): Метрики обучения
        validation_states (list): Метрики валидации
    '''
    torch.save(model, model_filepath)
    with open("data/train_states.json", "w", encoding="utf-8") as file:
        json.dump(train_states, file, indent=4, ensure_ascii=False)

    with open("data/validation_states.json", "w", encoding="utf-8") as file:
        json.dump(validation_states, file, indent=4, ensure_ascii=False)

In [13]:
def get_max_source_target_seq_len(df:pandas.DataFrame):
    source_max_len = target_max_len = -1
    for i in range(len(df)):
        source_max_len = max(source_max_len, len(df.loc[i, 'source_text']))
        target_max_len = max(target_max_len, len(df.loc[i, 'target_text']))
    return (source_max_len, target_max_len)

In [14]:
tokenizer = SeparatorTokenizer()

In [15]:
df = pd.read_csv(DATASET_FILEPATH)

In [16]:
df['target_text'] = ''
df = df.rename(columns={'name' : 'source_text'})
for i in range(len(df)):
    df.loc[i, 'ingredients'] = re.sub(r'\. ?\.\n', '.\n ', df.loc[i, 'ingredients'])
    df.loc[i, 'target_text'] = f'НАЗВАНИЕ: {df.loc[i, 'source_text']}.\n ИНГРЕДИЕНТЫ: {df.loc[i, 'ingredients']}.\n ИНСТРУКЦИЯ: {df.loc[i, 'Instructions']}.'
df = df.drop(columns=['Instructions', 'ingredients', 'composition_list'])

In [17]:
df['split'] = 'train'
selected_indices = df.sample(int(EVAL_PROPORTION*len(df)), random_state=RANDOM_STATE).index
df.loc[selected_indices, 'split'] = 'validation'

# К нижнему регистру, токенизация и очистка от служебных символов
df['source_text'] = df['source_text'].apply(lambda x: tokenizer.tokenize(x.lower()))
df['target_text'] = df['target_text'].apply(lambda x: tokenizer.tokenize(x.lower()))

In [18]:
source_max_len, target_max_len = get_max_source_target_seq_len(df)
print(source_max_len)
print(target_max_len)

14
1273


In [19]:
tokens_vocab = Vocabulary()

tokens_freq = get_tokens_freq(df)

for key, value in tokens_freq.items():
    if value > TOKENS_TRESHOLD_FREQ:
        tokens_vocab.add_token(key)

tokens_vocab.to_json('data/tokens_vocab_recipes.json')

In [20]:
tokens_vocab = Vocabulary.from_json('data/tokens_vocab_recipes.json')

In [21]:
vectorizer = Seq2Seq_Vectorizer(tokens_vocab, MAX_SOURCE_SEQ_LEN, MAX_TARGET_SEQ_LEN)
dataset = CustomDataset(df, tokenizer, vectorizer)

In [22]:
vocab_size = len(tokens_vocab)
mask_index = tokens_vocab.mask_token_index

In [23]:
if USE_PRETRAINED:
    with open("data/train_states.json", "r", encoding="utf-8") as file:
        train_states = json.load(file)

    with open("data/validation_states.json", "r", encoding="utf-8") as file:
        validation_states = json.load(file)
    
    model = torch.load(MODEL_SAVE_FILEPATH, weights_only=False)
else:
    train_states = []
    validation_states = []
    model = TransformerModel(vocab_size, vocab_size, EMBEDDING_DIM, MODEL_DIM, NUM_HEAD, NUM_ENCODER_LAYERS,\
                         NUM_DECODER_LAYERS, FC_HIDDEN_DIM, DROPOUT, MAX_SEQ_LEN, BATCH_FIRST, mask_index)

In [24]:
model = model.to(DEVICE)

optimizer = optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=LR_SCHEDULER_FACTOR, patience=LR_SCHEDULER_PATIENCE)

In [ ]:
try:
    for epoch in range(EPOCHS):
        dataset.set_dataframe_split('train')
        batch_generator = generate_batches(dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, drop_last=DROP_LAST, device=DEVICE)
        train_running_loss = 0.0
        train_running_acc = 0.0
        epoch_err = 0.0

        model.train()
        
        for batch_index, batch_dict in enumerate(batch_generator):

            optimizer.zero_grad()

            # Предсказание
            y_pred = model(batch_dict['source_vec'],
                           batch_dict['target_x_vec'],
                           apply_softmax = False,
                           temperature = TEMPERATURE)

            # потери
            loss = sequence_loss(y_pred, batch_dict['target_y_vec'], mask_index=mask_index)

            loss.backward()

            optimizer.step()

            # Средние потери и точность
            train_running_loss += (loss.item() - train_running_loss) / (batch_index + 1)
            epoch_err += loss.item()

            acc_t = compute_accuracy(y_pred, batch_dict['target_y_vec'], mask_index)
            train_running_acc += (acc_t - train_running_acc) / (batch_index + 1)

        train_states.append({'epoch' : epoch+1, 'epoch_loss' : epoch_err, 'epoch_running_loss' : train_running_loss, 'accuracy' : train_running_acc})

        print('-'*40)
        print(f'epoch {epoch+1}')
        print(f'train_epoch_error {epoch_err}')
        print(f'train loss {train_running_loss}   ,   train accuracy {train_running_acc}')


        if EVAL_PROPORTION > 0:
            dataset.set_dataframe_split('validation')
            batch_generator = generate_batches(dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, drop_last=DROP_LAST, device=DEVICE)
            valid_running_loss = 0.0
            valid_running_acc = 0.0
            epoch_err = 0.0

            model.eval()
            with torch.no_grad():
                for batch_index, batch_dict in enumerate(batch_generator):
                    # Предсказание
                    y_pred = model(batch_dict['source_vec'],
                                batch_dict['target_x_vec'],
                                apply_softmax = False,
                                temperature = TEMPERATURE)

                    # потери
                    loss = sequence_loss(y_pred, batch_dict['target_y_vec'], mask_index)

                    # Средние потери и точность
                    valid_running_loss += (loss.item() - valid_running_loss) / (batch_index + 1)
                    epoch_err += loss.item()

                    acc_t = compute_accuracy(y_pred, batch_dict['target_y_vec'], mask_index)
                    valid_running_acc += (acc_t - valid_running_acc) / (batch_index + 1)

            validation_states.append({'epoch' : epoch+1, 'epoch_loss' : epoch_err, 'epoch_running_loss' : valid_running_loss, 'accuracy' : valid_running_acc})

            scheduler.step(valid_running_loss)

            print(f'validation_epoch_error {epoch_err}')
            print(f'validation loss {valid_running_loss}   ,   validation accuracy {valid_running_acc}')
        else:
            scheduler.step(train_running_loss)
        
except KeyboardInterrupt:
    print("Аварийная остановка")

In [45]:
query = 'пончики с маслом'

In [46]:
query = query.lower()
indices = generate(model, tokenizer, vectorizer, query, max_seq_len=MAX_SEQ_LEN, temperature=TEMPERATURE, device=DEVICE)
response = decode_indices(indices, vectorizer)

In [47]:
response

['<BOS> название : творожные с вишней . ингредиенты : пшеничная мука 0 . 5 стак . вода 0 . 2 стак . сливочное масло 0 . 5 столов . яйцо куриное 0 . 5 шт . молоко 0 . 2 стак . сахар 0 . 2 стак . соль щепотка по_вкусу . . инструкция : 1 . смешать муку , соль , разрыхлитель и соль . 2 . в глубокой миске взбить яйца , всыпать муку и сахар . 3 . добавить молоко , пока не станет однородной . 4 . добавить в тесто лук и быстро вымесить тесто . 5 . выпекать в течение 30 - 40 минут . 6 . выложить на бумажное полотенце . . <EOS> ']

In [29]:
save_results_to_file(model, MODEL_SAVE_FILEPATH, train_states, validation_states)